In [77]:
import pandas as pd
import numpy as np

from pathlib import Path

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_fscore_support,
    mean_absolute_error, root_mean_squared_error
)

from xgboost import XGBClassifier, XGBRegressor


In [78]:
DATA_DIR = Path("../data/processed")
df = pd.read_csv(DATA_DIR / "dataset_entrenamiento_grid.csv")

print(df.shape)
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_15112\2930327741.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_DIR / "dataset_entrenamiento_grid.csv")


(2202849, 13)


,grid_id,fecha,conteo_delitos,codigo_parroquia,llamadas_totales,llamadas_contexto,llamadas_otro,llamadas_propiedad,llamadas_violencia,delitos_lag_1,delitos_7d,dia_semana,es_fin_semana
0,-100_-7774,2025-01-08,0.0,150156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
1,-100_-7774,2025-01-09,0.0,150156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0
2,-100_-7774,2025-01-10,0.0,150156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0
3,-100_-7774,2025-01-11,0.0,150156,1.0,0.0,1.0,0.0,0.0,0.0,0.0,5,1
4,-100_-7774,2025-01-12,0.0,150156,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6,1


In [79]:
df["fecha"] = pd.to_datetime(df["fecha"])

# grid_id como "gx_gy"
grid_split = df["grid_id"].astype(str).str.split("_", expand=True)
df["grid_x"] = grid_split[0].astype(int)
df["grid_y"] = grid_split[1].astype(int)

df[["grid_id", "grid_x", "grid_y"]].head()


,grid_id,grid_x,grid_y
0,-100_-7774,-100,-7774
1,-100_-7774,-100,-7774
2,-100_-7774,-100,-7774
3,-100_-7774,-100,-7774
4,-100_-7774,-100,-7774


In [80]:
df["codigo_parroquia"] = df["codigo_parroquia"].astype(str)

parr_freq = df["codigo_parroquia"].value_counts(normalize=True)
df["parroquia_freq"] = df["codigo_parroquia"].map(parr_freq).fillna(0)

df[["codigo_parroquia", "parroquia_freq"]].head()


,codigo_parroquia,parroquia_freq
0,150156,0.002697
1,150156,0.002697
2,150156,0.002697
3,150156,0.002697
4,150156,0.002697


In [81]:
df["y_bin"] = (df["conteo_delitos"] > 0).astype(int)


In [82]:
FEATURES = [
    # ubicación (grid)
    "grid_x", "grid_y",
    # contexto parroquial
    "parroquia_freq",
    # ECU911 contexto
    "llamadas_totales",
    "llamadas_contexto",
    "llamadas_propiedad",
    "llamadas_violencia",
    "llamadas_otro",
    # historial delito
    "delitos_lag_1",
    "delitos_7d",
    # tiempo
    "dia_semana",
    "es_fin_semana"
]

# Asegurar que existan
missing = [c for c in FEATURES if c not in df.columns]
print("Missing features:", missing)

X = df[FEATURES].copy()
y_bin = df["y_bin"].copy()
y_count = df["conteo_delitos"].copy()


Missing features: []


In [83]:
# Orden temporal
df_sorted = df.sort_values("fecha").reset_index(drop=True)

unique_dates = df_sorted["fecha"].sort_values().unique()
cut_idx = int(len(unique_dates) * 0.8)  # 80% train, 20% test
cut_date = unique_dates[cut_idx]

train_mask = df_sorted["fecha"] < cut_date
test_mask  = df_sorted["fecha"] >= cut_date

X_train = df_sorted.loc[train_mask, FEATURES]
X_test  = df_sorted.loc[test_mask, FEATURES]

ybin_train = df_sorted.loc[train_mask, "y_bin"]
ybin_test  = df_sorted.loc[test_mask, "y_bin"]

ycount_train = df_sorted.loc[train_mask, "conteo_delitos"]
ycount_test  = df_sorted.loc[test_mask, "conteo_delitos"]

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Cut date:", cut_date)


Train: (1757829, 12) Test: (445020, 12)
Cut date: 2025-09-02 00:00:00


In [84]:
# Fechas únicas ordenadas
dates = np.sort(df_sorted["fecha"].unique())

cut_train = int(len(dates) * 0.7)
cut_val   = int(len(dates) * 0.8)

train_dates = dates[:cut_train]
val_dates   = dates[cut_train:cut_val]
test_dates  = dates[cut_val:]

train_mask = df_sorted["fecha"].isin(train_dates)
val_mask   = df_sorted["fecha"].isin(val_dates)
test_mask  = df_sorted["fecha"].isin(test_dates)

X_train, y_train = df_sorted.loc[train_mask, FEATURES], df_sorted.loc[train_mask, "y_bin"]
X_val,   y_val   = df_sorted.loc[val_mask, FEATURES],   df_sorted.loc[val_mask, "y_bin"]
X_test,  y_test  = df_sorted.loc[test_mask, FEATURES],  df_sorted.loc[test_mask, "y_bin"]

print(X_train.shape, X_val.shape, X_test.shape)
print("Prevalencia train:", y_train.mean(), "val:", y_val.mean(), "test:", y_test.mean())


(1535319, 12) (222510, 12) (445020, 12)
Prevalencia train: 0.009204601779825562 val: 0.009401824637094961 test: 0.008992854253741404


In [85]:
pos = y_train.sum()
neg = len(y_train) - pos
spw = np.sqrt(neg / max(pos, 1))  # <-- suavizado (clave)

pos_rate = float(y_train.mean())

clf = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=5,
    min_child_weight=10,
    gamma=1.0,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=2.0,
    reg_alpha=0.0,
    objective="binary:logistic",
    eval_metric="aucpr",
    scale_pos_weight=spw,
    base_score=pos_rate,        # <-- importante para rare events
    max_delta_step=1,           # <-- estabiliza en desbalance extremo
    random_state=42,
    n_jobs=-1
)

clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)


,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",0.009204601779825562
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.8
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import

In [108]:
p_test = clf.predict_proba(X_test)[:, 1]

roc = roc_auc_score(ybin_test, p_test)
aucpr = average_precision_score(ybin_test, p_test)

print("Etapa 1 - ROC AUC:", roc)
print("Etapa 1 - PR AUC :", aucpr)


Etapa 1 - ROC AUC: 0.8163609343988604
Etapa 1 - PR AUC : 0.06601600180882602


In [109]:
# threshold = 0.8
# y_pred_bin = (p_test >= threshold).astype(int)

# prec, rec, f1, _ = precision_recall_fscore_support(
#     ybin_test, y_pred_bin, average="binary", zero_division=0
# )

# print("Threshold:", threshold)
# print("Precision:", prec, "Recall:", rec, "F1:", f1)


In [110]:
from sklearn.linear_model import LogisticRegression

# Probabilidades crudas en validación
p_val_raw = clf.predict_proba(X_val)[:, 1]
p_val_raw = np.clip(p_val_raw, 1e-6, 1-1e-6)

# Logit transform
logit_val = np.log(p_val_raw / (1 - p_val_raw)).reshape(-1, 1)

# Calibrador (Platt scaling)
cal = LogisticRegression(solver="lbfgs")
cal.fit(logit_val, y_val)

# Probabilidades calibradas en test
p_test_raw = clf.predict_proba(X_test)[:, 1]
p_test_raw = np.clip(p_test_raw, 1e-6, 1-1e-6)
logit_test = np.log(p_test_raw / (1 - p_test_raw)).reshape(-1, 1)

p_test = cal.predict_proba(logit_test)[:, 1]

print("p_test CAL stats:",
      "min=", float(p_test.min()),
      "mean=", float(p_test.mean()),
      "median=", float(np.median(p_test)),
      "max=", float(p_test.max()))
print("Prevalencia real test:", float(y_test.mean()))


p_test CAL stats: min= 5.8096247644948596e-05 mean= 0.009528966444338184 median= 0.0036893738283070446 max= 0.5446190876103961
Prevalencia real test: 0.008992854253741404


In [111]:
from sklearn.metrics import precision_recall_curve

# Probabilidades calibradas en VALIDACIÓN
p_val_raw = clf.predict_proba(X_val)[:, 1]
p_val_raw = np.clip(p_val_raw, 1e-6, 1-1e-6)
logit_val = np.log(p_val_raw / (1 - p_val_raw)).reshape(-1, 1)
p_val = cal.predict_proba(logit_val)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_val, p_val)

precision = precision[:-1]
recall = recall[:-1]

f1 = 2 * (precision * recall) / (precision + recall + 1e-12)

best_idx = np.argmax(f1)
best_thr_f1 = thresholds[best_idx]

print("Best threshold (F1):", float(best_thr_f1))
print("Precision:", float(precision[best_idx]))
print("Recall:", float(recall[best_idx]))
print("F1:", float(f1[best_idx]))

pred_val = (p_val >= best_thr_f1).astype(int)
print("Alert rate (val):", float(pred_val.mean()))


Best threshold (F1): 0.0699471175504657
Precision: 0.10885245901639344
Recall: 0.23804971319311663
F1: 0.14939253037305067
Alert rate (val): 0.020560873668599162


In [120]:
pred_val = (p_val >= best_thr_f1).astype(int)
alert_rate = pred_val.mean()

print("Tasa de alertas (val):", alert_rate)
print("Número de celdas marcadas:", pred_val.sum(), "de", len(pred_val))


Tasa de alertas (val): 0.020560873668599162
Número de celdas marcadas: 4575 de 222510


In [113]:
train_pos = df_sorted.loc[train_mask & (df_sorted["conteo_delitos"] > 0)]
val_pos   = df_sorted.loc[val_mask   & (df_sorted["conteo_delitos"] > 0)]
test_pos  = df_sorted.loc[test_mask  & (df_sorted["conteo_delitos"] > 0)]

X2_train = train_pos[FEATURES]
y2_train = train_pos["conteo_delitos"]

X2_test = test_pos[FEATURES]
y2_test = test_pos["conteo_delitos"]

print("Train positivos:", X2_train.shape, "Test positivos:", X2_test.shape)

reg = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=5,
    min_child_weight=10,
    gamma=1.0,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=2.0,
    objective="count:poisson",
    eval_metric="rmse",
    random_state=42,
    n_jobs=-1
)

reg.fit(
    train_pos[FEATURES], train_pos["conteo_delitos"],
    eval_set=[(val_pos[FEATURES], val_pos["conteo_delitos"])],
    verbose=False
)

lambda_test_all = np.clip(reg.predict(X_test), 0, None)
expected_count = p_test * lambda_test_all

print("Promedio real conteo (test):", float(df_sorted.loc[test_mask, "conteo_delitos"].mean()))
print("Promedio esperado (test):", float(expected_count.mean()))

Train positivos: (14132, 12) Test positivos: (4002, 12)
Promedio real conteo (test): 0.013340973439395982
Promedio esperado (test): 0.013915867699795118


In [114]:
lambda_test_pos = reg.predict(X2_test)
lambda_test_pos = np.clip(lambda_test_pos, 0, None)

mae = mean_absolute_error(y2_test, lambda_test_pos)
rmse = root_mean_squared_error(y2_test, lambda_test_pos)

print("Etapa 2 - MAE (solo positivos):", mae)
print("Etapa 2 - RMSE (solo positivos):", rmse)


Etapa 2 - MAE (solo positivos): 0.6698130336718104
Etapa 2 - RMSE (solo positivos): 1.3407097074136314


In [115]:
lambda_test_all = reg.predict(X_test)
lambda_test_all = np.clip(lambda_test_all, 0, None)

expected_count = p_test * lambda_test_all


In [116]:
mae_all = mean_absolute_error(ycount_test, expected_count)
rmse_all = root_mean_squared_error(ycount_test, expected_count)

print("Global - MAE (E[conteo]):", mae_all)
print("Global - RMSE (E[conteo]):", rmse_all)

# Extra: ¿cuántos ceros predice en promedio?
print("Promedio y real:", ycount_test.mean())
print("Promedio esperado:", expected_count.mean())


Global - MAE (E[conteo]): 0.026081394119150038
Global - RMSE (E[conteo]): 0.1873504554260819
Promedio y real: 0.013340973439395982
Promedio esperado: 0.013915867699795118


In [117]:
q1, q2 = np.quantile(expected_count, [0.7, 0.9])  # ajustable

def risk_bucket(x):
    if x >= q2:
        return "ALTO"
    if x >= q1:
        return "MEDIO"
    return "BAJO"

risk_level = pd.Series(expected_count).apply(risk_bucket)

out = df_sorted.loc[test_mask, ["grid_id", "fecha", "codigo_parroquia"]].copy()
out["p_ocurre"] = p_test
out["lambda"] = lambda_test_all
out["expected_count"] = expected_count
out["riesgo"] = risk_level

out.head()


,grid_id,fecha,codigo_parroquia,p_ocurre,lambda,expected_count,riesgo
1757829,-270_-8025,2025-09-02,90156,0.009327,1.563047,0.014579,NaN
1757830,-426_-7923,2025-09-02,110161,0.001375,1.267077,0.001743,NaN
1757831,-91_-7781,2025-09-02,150350,0.008776,1.313787,0.011529,NaN
1757832,8_-7696,2025-09-02,210158,0.003023,1.360612,0.004113,NaN
1757833,-146_-7800,2025-09-02,160150,0.011243,1.353312,0.015215,NaN


In [118]:
print("p_test stats:",
      "min=", float(p_test.min()),
      "mean=", float(p_test.mean()),
      "median=", float(np.median(p_test)),
      "max=", float(p_test.max()))


p_test stats: min= 5.8096247644948596e-05 mean= 0.009528966444338184 median= 0.0036893738283070446 max= 0.5446190876103961


In [119]:
print("lambda_test_all stats:",
      "min=", float(lambda_test_all.min()),
      "mean=", float(lambda_test_all.mean()),
      "median=", float(np.median(lambda_test_all)),
      "max=", float(lambda_test_all.max()))


lambda_test_all stats: min= 1.0347977876663208 mean= 1.389418125152588 median= 1.364577293395996 max= 6.5477495193481445
